## 1. Setup iniziale

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_cerebras import ChatCerebras

model = ChatCerebras(model="gpt-oss-120b", temperature=0.7)

## 2. Contesto e Runtime

In [ ]:
#from typing import TypedDict
from langchain.agents import create_agent
from typing import NotRequired, TypedDict

class Context(TypedDict):
    segreto: NotRequired[str]

In [ ]:
from langchain.tools import tool

@tool
def get_secret_value_from_context(context: Context) -> str:
    "Ritorna il valore segreto dal contesto"
    print("Context received in tool:", context)
    for key, value in context.items():
        print(f"Key: {key}, Value: {value}")

    if not isinstance(context, dict):
        secret = getattr(context, "segreto", "Nessun segreto trovato.")  
    else:
        secret = context.get("segreto", "Nessun segreto trovato.")
    
    return secret

In [ ]:
agent = create_agent(
    model=model,
    context_schema=Context,
    tools=[get_secret_value_from_context],
    system_prompt="Sei un agente che può memorizzare e recuperare un valore segreto a discrezione dell'utente. Se ti viene chiesto il valore segreto, usa lo strumento per recuperarlo.",
)

In [ ]:
agent.invoke({"messages": "Qual è il valore segreto?"}, context=Context(segreto="5864-ABCD-1234-EFGH"))

### Accedere al contesto

In [ ]:
from langchain.tools import ToolRuntime

@tool
def get_secret_value_from_runtime_context(runtime: ToolRuntime) -> str:
    "Ritorna il valore segreto dal contesto dell'agente."
    #-----DEBUG-----
    print("Runtime received in tool:")
    for attr in dir(runtime):
        if not attr.startswith("_"):
            print(f" - {attr}: {getattr(runtime, attr)}")

    # Check if context exists and has the required key
    if runtime.context is None:
        return "Error: runtime.context is None"
    
    if 'segreto' not in runtime.context:
        return f"Error: 'segreto' key not found in context. Available keys: {list(runtime.context.keys())}"
    
    return runtime.context['segreto']

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    context_schema=Context,
    tools=[get_secret_value_from_runtime_context],
    checkpointer=InMemorySaver(),
    system_prompt="Sei un agente che può memorizzare e recuperare un valore segreto a discrezione dell'utente. Se ti viene chiesto il valore segreto, usa lo strumento per recuperarlo."
)

In [ ]:
from uuid import uuid4

config = { "configurable": { "thread_id": str(uuid4()) } }

agent.invoke({"messages": "Qual è il valore segreto?"}, context=Context(segreto="25685"), config=config)

## 3. Stato e Runtime

In [ ]:
from langchain.agents import AgentState

class CustomState(AgentState):
    segreto: NotRequired[str]

In [ ]:
@tool
def get_secret_value_state(runtime: ToolRuntime) -> str:
    "Ritorna il valore segreto dallo stato dell'agente"
    for key, value in runtime.state.items():
        print(f" - {key}: {value}")
    return runtime.state.get('segreto', 'Segreto non disponibile') 

In [ ]:
# Modificare lo stato all'interno del tool modifica lo stato solo LOCALMENTE per quel tool invocation

@tool
def set_secret_value_state(runtime: ToolRuntime, secret: str) -> str:
    "Imposta il valore segreto nello stato dell'agente."
    runtime.state['segreto'] = secret
    for key, value in runtime.state.items():
        print(f" - {key}: {value}")
    return f"Valore segreto impostato correttamente"

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    system_prompt="Sei un agente che può memorizzare e recuperare un valore segreto a discrezione dell'utente. Se ti viene chiesto il valore segreto, usa lo strumento per recuperarlo." \
    "Se ti viene chiesto di impostare il valore segreto, usa lo strumento per impostarlo nello stato dell'agente.",
    state_schema=CustomState,
    tools=[get_secret_value_state, set_secret_value_state],
    checkpointer=InMemorySaver(),
    
)

In [ ]:
from uuid import uuid4

config = { "configurable": { "thread_id": str(uuid4()) } }

response = agent.invoke({"messages": "Qual è il valore segreto?"}, config=config)

In [ ]:
for r in response['messages']:
    r.pretty_print()

In [ ]:
agent.invoke({"messages": "Setta il valore segreto a 12345"}, config=config)['messages'][-1].pretty_print()

In [ ]:
response = agent.invoke({"messages": "Qual è il valore segreto?"}, config=config)['messages']

for msg in response:
    msg.pretty_print()

L'agente **può** riuscire a rispondere ma solo perché ha la memoria della conversazione, il tool restituisce: "Segreto non disponibile" <br>
Il modo corretto di aggiornare lo stato è utilizzare Command.

In [ ]:
# Per aggiornare correttamente lo stato va usato Command (applica correttamente il reducer allo stato globale)

from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def set_secret_value_state(runtime: ToolRuntime, secret: str) -> Command:
    "Imposta il valore segreto nello stato dell'agente."
    # Command viene intercettato da LangGraph che applica correttamente l'aggiornamento allo stato globale
    return Command(
        update={"segreto": secret,                                                                                              # Aggiorna lo stato custom
                "messages": [ToolMessage(content=f"Valore segreto impostato a: {secret}", tool_call_id=runtime.tool_call_id)]}  # Registra la risposta nello storico dei messaggi
    )

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    state_schema=CustomState,
    tools=[get_secret_value_state, set_secret_value_state],
    checkpointer=InMemorySaver(),
    system_prompt="Sei un agente che può memorizzare e recuperare un valore segreto a discrezione dell'utente. Se ti viene chiesto il valore segreto, usa lo strumento per settarlo e recuperarlo."
)

In [ ]:
from uuid import uuid4

config = { "configurable": { "thread_id": str(uuid4()) } }

agent.invoke({"messages": "Qual è il valore segreto?"}, config=config)['messages'][-1].pretty_print()

In [ ]:
agent.invoke({"messages": "Setta il valore segreto a 12345"}, config=config)['messages'][-1].pretty_print()

In [ ]:
response = agent.invoke({"messages": "Qual è il valore segreto?"}, config=config)['messages']
for msg in response:
    msg.pretty_print()